# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [2]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for the bonus section.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [3]:
from cs231n.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [ ]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

# learning_rates = [1e-9, 1e-8, 1e-7]
# regularization_strengths = [5e4, 5e5, 5e6]
learning_rates = np.linspace(1e-11, 1e-7, 20)
regularization_strengths = np.linspace(5e2, 5e6, 20)

results = {}
best_val = -1
best_svm = None


################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
for lr in learning_rates:
    for reg in regularization_strengths:
        svm = LinearSVM()
        svm.train(X_train_feats, y_train, learning_rate=lr, reg=reg, num_iters=6000, verbose=True)
        y_train_pred = svm.predict(X_train_feats)
        y_val_pred = svm.predict(X_val_feats)
        train_acc = np.mean(y_train == y_train_pred)
        val_acc = np.mean(y_val == y_val_pred)
        results[(lr, reg)] = (train_acc, val_acc)
        if val_acc > best_val:
            best_val = val_acc
            best_svm = svm
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

iteration 0 / 6000: loss 9.382765
iteration 100 / 6000: loss 9.403745
iteration 200 / 6000: loss 9.389956
iteration 300 / 6000: loss 9.389993
iteration 400 / 6000: loss 9.399023
iteration 500 / 6000: loss 9.404988
iteration 600 / 6000: loss 9.398943
iteration 700 / 6000: loss 9.396612
iteration 800 / 6000: loss 9.397518
iteration 900 / 6000: loss 9.409600
iteration 1000 / 6000: loss 9.391475
iteration 1100 / 6000: loss 9.374940
iteration 1200 / 6000: loss 9.397342
iteration 1300 / 6000: loss 9.385092
iteration 1400 / 6000: loss 9.387777
iteration 1500 / 6000: loss 9.413424
iteration 1600 / 6000: loss 9.388753
iteration 1700 / 6000: loss 9.395393
iteration 1800 / 6000: loss 9.393226
iteration 1900 / 6000: loss 9.378356
iteration 2000 / 6000: loss 9.397241
iteration 2100 / 6000: loss 9.398069
iteration 2200 / 6000: loss 9.396869
iteration 2300 / 6000: loss 9.397757
iteration 2400 / 6000: loss 9.384874
iteration 2500 / 6000: loss 9.377057
iteration 2600 / 6000: loss 9.392303
iteration 270

iteration 3500 / 6000: loss 592.669721
iteration 3600 / 6000: loss 591.743909
iteration 3700 / 6000: loss 590.818300
iteration 3800 / 6000: loss 589.890968
iteration 3900 / 6000: loss 588.976703
iteration 4000 / 6000: loss 588.063987
iteration 4100 / 6000: loss 587.162204
iteration 4200 / 6000: loss 586.249027
iteration 4300 / 6000: loss 585.317938
iteration 4400 / 6000: loss 584.412101
iteration 4500 / 6000: loss 583.512689
iteration 4600 / 6000: loss 582.611083
iteration 4700 / 6000: loss 581.701019
iteration 4800 / 6000: loss 580.796762
iteration 4900 / 6000: loss 579.895309
iteration 5000 / 6000: loss 579.000657
iteration 5100 / 6000: loss 578.093369
iteration 5200 / 6000: loss 577.192330
iteration 5300 / 6000: loss 576.309118
iteration 5400 / 6000: loss 575.389580
iteration 5500 / 6000: loss 574.509871
iteration 5600 / 6000: loss 573.628894
iteration 5700 / 6000: loss 572.728046
iteration 5800 / 6000: loss 571.850739
iteration 5900 / 6000: loss 570.939985
iteration 0 / 6000: loss 

iteration 500 / 6000: loss 1396.417340
iteration 600 / 6000: loss 1391.324245
iteration 700 / 6000: loss 1386.248376
iteration 800 / 6000: loss 1381.187391
iteration 900 / 6000: loss 1376.127112
iteration 1000 / 6000: loss 1371.084323
iteration 1100 / 6000: loss 1366.086923
iteration 1200 / 6000: loss 1361.097641
iteration 1300 / 6000: loss 1356.124135
iteration 1400 / 6000: loss 1351.162037
iteration 1500 / 6000: loss 1346.239685
iteration 1600 / 6000: loss 1341.315614
iteration 1700 / 6000: loss 1336.402517
iteration 1800 / 6000: loss 1331.537466
iteration 1900 / 6000: loss 1326.665590
iteration 2000 / 6000: loss 1321.821204
iteration 2100 / 6000: loss 1316.981013
iteration 2200 / 6000: loss 1312.163910
iteration 2300 / 6000: loss 1307.389313
iteration 2400 / 6000: loss 1302.588192
iteration 2500 / 6000: loss 1297.860507
iteration 2600 / 6000: loss 1293.114704
iteration 2700 / 6000: loss 1288.388772
iteration 2800 / 6000: loss 1283.667378
iteration 2900 / 6000: loss 1278.980381
itera

iteration 3500 / 6000: loss 1701.840299
iteration 3600 / 6000: loss 1692.953352
iteration 3700 / 6000: loss 1684.104818
iteration 3800 / 6000: loss 1675.310583
iteration 3900 / 6000: loss 1666.555041
iteration 4000 / 6000: loss 1657.871495
iteration 4100 / 6000: loss 1649.211355
iteration 4200 / 6000: loss 1640.598861
iteration 4300 / 6000: loss 1632.031944
iteration 4400 / 6000: loss 1623.497444
iteration 4500 / 6000: loss 1615.027707
iteration 4600 / 6000: loss 1606.587220
iteration 4700 / 6000: loss 1598.221048
iteration 4800 / 6000: loss 1589.882456
iteration 4900 / 6000: loss 1581.574941
iteration 5000 / 6000: loss 1573.311813
iteration 5100 / 6000: loss 1565.098952
iteration 5200 / 6000: loss 1556.936677
iteration 5300 / 6000: loss 1548.813976
iteration 5400 / 6000: loss 1540.724108
iteration 5500 / 6000: loss 1532.665490
iteration 5600 / 6000: loss 1524.680320
iteration 5700 / 6000: loss 1516.733083
iteration 5800 / 6000: loss 1508.805497
iteration 5900 / 6000: loss 1500.928657


iteration 300 / 6000: loss 2779.026752
iteration 400 / 6000: loss 2758.687380
iteration 500 / 6000: loss 2738.503094
iteration 600 / 6000: loss 2718.454316
iteration 700 / 6000: loss 2698.584014
iteration 800 / 6000: loss 2678.815301
iteration 900 / 6000: loss 2659.215441
iteration 1000 / 6000: loss 2639.768542
iteration 1100 / 6000: loss 2620.447511
iteration 1200 / 6000: loss 2601.274348
iteration 1300 / 6000: loss 2582.232352
iteration 1400 / 6000: loss 2563.334193
iteration 1500 / 6000: loss 2544.593673
iteration 1600 / 6000: loss 2525.977362
iteration 1700 / 6000: loss 2507.507717
iteration 1800 / 6000: loss 2489.166414
iteration 1900 / 6000: loss 2470.964936
iteration 2000 / 6000: loss 2452.876119
iteration 2100 / 6000: loss 2434.939964
iteration 2200 / 6000: loss 2417.105341
iteration 2300 / 6000: loss 2399.447001
iteration 2400 / 6000: loss 2381.884679
iteration 2500 / 6000: loss 2364.474885
iteration 2600 / 6000: loss 2347.183515
iteration 2700 / 6000: loss 2330.009169
iterati

...

iteration 500 / 6000: loss 8.999895
iteration 600 / 6000: loss 8.999858
iteration 700 / 6000: loss 8.999890
iteration 800 / 6000: loss 8.999860
iteration 900 / 6000: loss 8.999873
iteration 1000 / 6000: loss 8.999863
iteration 1100 / 6000: loss 8.999879
iteration 1200 / 6000: loss 8.999847
iteration 1300 / 6000: loss 8.999911
iteration 1400 / 6000: loss 8.999877
iteration 1500 / 6000: loss 8.999873
iteration 1600 / 6000: loss 8.999870
iteration 1700 / 6000: loss 8.999877
iteration 1800 / 6000: loss 8.999878
iteration 1900 / 6000: loss 8.999874
iteration 2000 / 6000: loss 8.999901
iteration 2100 / 6000: loss 8.999886
iteration 2200 / 6000: loss 8.999861
iteration 2300 / 6000: loss 8.999866
iteration 2400 / 6000: loss 8.999867
iteration 2500 / 6000: loss 8.999855
iteration 2600 / 6000: loss 8.999877
iteration 2700 / 6000: loss 8.999855
iteration 2800 / 6000: loss 8.999857
iteration 2900 / 6000: loss 8.999849
iteration 3000 / 6000: loss 8.999855
iteration 3100 / 6000: loss 8.999883
iterat

iteration 5000 / 6000: loss 8.999967
iteration 5100 / 6000: loss 8.999966
iteration 5200 / 6000: loss 8.999974
iteration 5300 / 6000: loss 8.999962
iteration 5400 / 6000: loss 8.999971
iteration 5500 / 6000: loss 8.999965
iteration 5600 / 6000: loss 8.999964
iteration 5700 / 6000: loss 8.999973
iteration 5800 / 6000: loss 8.999970
iteration 5900 / 6000: loss 8.999962
iteration 0 / 6000: loss 1063.309072
iteration 100 / 6000: loss 8.999972
iteration 200 / 6000: loss 8.999969
iteration 300 / 6000: loss 8.999975
iteration 400 / 6000: loss 8.999973
iteration 500 / 6000: loss 8.999969
iteration 600 / 6000: loss 8.999973
iteration 700 / 6000: loss 8.999984
iteration 800 / 6000: loss 8.999976
iteration 900 / 6000: loss 8.999974
iteration 1000 / 6000: loss 8.999972
iteration 1100 / 6000: loss 8.999979
iteration 1200 / 6000: loss 8.999980
iteration 1300 / 6000: loss 8.999972
iteration 1400 / 6000: loss 8.999979
iteration 1500 / 6000: loss 8.999975
iteration 1600 / 6000: loss 8.999972
iteration 

iteration 3500 / 6000: loss 8.999984
iteration 3600 / 6000: loss 8.999986
iteration 3700 / 6000: loss 8.999982
iteration 3800 / 6000: loss 8.999984
iteration 3900 / 6000: loss 8.999984
iteration 4000 / 6000: loss 8.999985
iteration 4100 / 6000: loss 8.999990
iteration 4200 / 6000: loss 8.999983
iteration 4300 / 6000: loss 8.999984
iteration 4400 / 6000: loss 8.999987
iteration 4500 / 6000: loss 8.999985
iteration 4600 / 6000: loss 8.999989
iteration 4700 / 6000: loss 8.999983
iteration 4800 / 6000: loss 8.999987
iteration 4900 / 6000: loss 8.999985
iteration 5000 / 6000: loss 8.999984
iteration 5100 / 6000: loss 8.999985
iteration 5200 / 6000: loss 8.999981
iteration 5300 / 6000: loss 8.999983
iteration 5400 / 6000: loss 8.999988
iteration 5500 / 6000: loss 8.999985
iteration 5600 / 6000: loss 8.999986
iteration 5700 / 6000: loss 8.999982
iteration 5800 / 6000: loss 8.999984
iteration 5900 / 6000: loss 8.999986
iteration 0 / 6000: loss 1833.450610
iteration 100 / 6000: loss 8.999990
it

iteration 2000 / 6000: loss 8.999992
iteration 2100 / 6000: loss 8.999989
iteration 2200 / 6000: loss 8.999991
iteration 2300 / 6000: loss 8.999990
iteration 2400 / 6000: loss 8.999991
iteration 2500 / 6000: loss 8.999991
iteration 2600 / 6000: loss 8.999991
iteration 2700 / 6000: loss 8.999994
iteration 2800 / 6000: loss 8.999991
iteration 2900 / 6000: loss 8.999991
iteration 3000 / 6000: loss 8.999992
iteration 3100 / 6000: loss 8.999989
iteration 3200 / 6000: loss 8.999990
iteration 3300 / 6000: loss 8.999991
iteration 3400 / 6000: loss 8.999991
iteration 3500 / 6000: loss 8.999990
iteration 3600 / 6000: loss 8.999990
iteration 3700 / 6000: loss 8.999992
iteration 3800 / 6000: loss 8.999990
iteration 3900 / 6000: loss 8.999991
iteration 4000 / 6000: loss 8.999990
iteration 4100 / 6000: loss 8.999994
iteration 4200 / 6000: loss 8.999990
iteration 4300 / 6000: loss 8.999989
iteration 4400 / 6000: loss 8.999993
iteration 4500 / 6000: loss 8.999990
iteration 4600 / 6000: loss 8.999990
i

iteration 500 / 6000: loss 8.999994
iteration 600 / 6000: loss 8.999994
iteration 700 / 6000: loss 8.999993
iteration 800 / 6000: loss 8.999992
iteration 900 / 6000: loss 8.999993
iteration 1000 / 6000: loss 8.999996
iteration 1100 / 6000: loss 8.999995
iteration 1200 / 6000: loss 8.999994
iteration 1300 / 6000: loss 8.999995
iteration 1400 / 6000: loss 8.999994
iteration 1500 / 6000: loss 8.999994
iteration 1600 / 6000: loss 8.999992
iteration 1700 / 6000: loss 8.999993
iteration 1800 / 6000: loss 8.999996
iteration 1900 / 6000: loss 8.999994
iteration 2000 / 6000: loss 8.999995
iteration 2100 / 6000: loss 8.999993
iteration 2200 / 6000: loss 8.999994
iteration 2300 / 6000: loss 8.999995
iteration 2400 / 6000: loss 8.999992
iteration 2500 / 6000: loss 8.999993
iteration 2600 / 6000: loss 8.999994
iteration 2700 / 6000: loss 8.999994
iteration 2800 / 6000: loss 8.999993
iteration 2900 / 6000: loss 8.999995
iteration 3000 / 6000: loss 8.999994
iteration 3100 / 6000: loss 8.999994
iterat

In [ ]:
# Evaluate your trained SVM on the test set
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
Describe the misclassification results that you see. Do they make sense?

## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [ ]:
print(X_train_feats.shape)

In [ ]:
from cs231n.classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

In [ ]:
# Run your neural net classifier on the test set. You should be able to
# get more than 55% accuracy.

test_acc = (net.predict(X_test_feats) == y_test).mean()
print(test_acc)

# Bonus: Design your own features!

You have seen that simple image features can improve classification performance. So far we have tried HOG and color histograms, but other types of features may be able to achieve even better classification performance.

For bonus points, design and implement a new type of feature and use it for image classification on CIFAR-10. Explain how your feature works and why you expect it to be useful for image classification. Implement it in this notebook, cross-validate any hyperparameters, and compare its performance to the HOG + Color histogram baseline.

# Bonus: Do something extra!
Use the material and code we have presented in this assignment to do something interesting. Was there another question we should have asked? Did any cool ideas pop into your head as you were working on the assignment? This is your chance to show off!